In [2]:
!pwd

/root


In [3]:
! pip install --quiet git-remote-codecommit


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import boto3
import sagemaker
import time
from time import strftime

boto_session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=boto_session)
sm_client = boto3.client("sagemaker")
region = boto_session.region_name
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
account = sagemaker_session.boto_session.client("sts").get_caller_identity()["Account"]


print(f"account: {account}")
print(f"bucket: {bucket}")
print(f"region: {region}")
print(f"role: {role}")

account: 441249477288
bucket: sagemaker-ap-south-1-441249477288
region: ap-south-1
role: arn:aws:iam::441249477288:role/service-role/AmazonSageMaker-ExecutionRole-20221207T104621


In [5]:
from sagemaker.pytorch.processing import PyTorchProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role

In [6]:
input_dataset = "s3://sagemaker-ap-south-1-441249477288/dataset/intel.zip"

In [7]:
dvc_repo_url = "codecommit::ap-south-1://sagemaker-intel"
dvc_branch = "processed-dataset"

# The following block will take the code from EFS storage, zip and store to S3. The specified docker image is pulled from ECR with code from S3 and served

In [10]:
# pytorch_processor = PyTorchProcessor(
#     image_uri="441249477288.dkr.ecr.ap-south-1.amazonaws.com/sagemaker-training",
#     framework_version='1.12',
#     role=get_execution_role(),
#     instance_type='ml.t3.medium',
#     instance_count=1,
#     base_job_name='preprocess-intel-dataset',
#     env={
#         "DVC_REPO_URL": dvc_repo_url,
#         "DVC_BRANCH": dvc_branch,
#         "GIT_USER": "gokul-pv",
#         "GIT_EMAIL": "25975535+gokul-pv@users.noreply.github.com"
#     }
# )

In [ ]:
# pytorch_processor.run(
#     code="AWS-SageMaker/preprocess.py",
#     inputs=[
#         ProcessingInput(
#             input_name='data',
#             source=input_dataset,
#             destination='/opt/ml/processing/input'
#         )
#     ],
# )

# The following block will take the specified docker image from ECR and run the code which is inside the docker

In [8]:
from sagemaker.processing import Processor

In [9]:
pytorch_processor = Processor(
    role=get_execution_role(),
    image_uri="441249477288.dkr.ecr.ap-south-1.amazonaws.com/sagemaker-training",
    instance_count=1,
    # instance_type="local",
    instance_type='ml.t3.medium',
    entrypoint = ["python", "code/preprocess.py"],
    base_job_name='preprocess-intel-dataset',
    env={
        "DVC_REPO_URL": dvc_repo_url,
        "DVC_BRANCH": dvc_branch,
        "GIT_USER": "gokul-pv",
        "GIT_EMAIL": "25975535+gokul-pv@users.noreply.github.com"
    }
)

In [10]:
pytorch_processor.run(
    inputs=[
        ProcessingInput(
            input_name='data',
            source=input_dataset,
            destination='/opt/ml/processing/input'
        )
    ],
)


Job Name:  preprocess-intel-dataset-2022-12-14-06-38-04-974
Inputs:  [{'InputName': 'data', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-south-1-441249477288/dataset/intel.zip', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  []
...........................................................................................................................................................:: Configuring Git
#011:: Cloning repo: codecommit::ap-south-1://sagemaker-intel
:: Generate Train Test Split
:: Extracting Zip /opt/ml/processing/input/intel.zip to /opt/ml/processing/tmp
#011:: Creating Directory /opt/ml/processing/sagemaker-intel/dataset/train
#011:: Creating Directory /opt/ml/processing/sagemaker-intel/dataset/test
:: Saving Datasets
:: Sync Processed Data to Git & DVC
:: Create branch processed-dataset
#011:: Create a new branch: processed-datas